In [ ]:
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
im = plt.imread("/content/drive/MyDrive/Colab Notebooks/image1.jpg")

In [ ]:
shape = np.shape(im)
shape

(1080, 1920, 3)

In [ ]:
@cuda.jit
def HSVconversion(input, hsv):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(8, 8, 3), dtype=np.uint8)
  
  if local_tidx < input.shape[0] and local_tidy < input.shape[1]:
    shared_memory[local_tidx, local_tidy, 0] = input[tidx, tidy, 0]
    shared_memory[local_tidx, local_tidy, 1] = input[tidx, tidy, 1]
    shared_memory[local_tidx, local_tidy, 2] = input[tidx, tidy, 2]
  
  cuda.syncthreads()

  R = shared_memory[local_tidx, local_tidy, 0]/255
  G = shared_memory[local_tidx, local_tidy, 1]/255
  B = shared_memory[local_tidx, local_tidy, 2]/255

  Max = max(R,G,B)

  hsv[tidx, tidy, 0] = Max
  


In [ ]:
@cuda.jit
def addpadding(input, output, omega):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(8, 8, 1), dtype=np.float64)
  
  if local_tidx < input.shape[0] and local_tidy < input.shape[1]:
    shared_memory[local_tidx, local_tidy, 0] = input[tidx, tidy, 0]

  cuda.syncthreads()

  if (tidx < omega) or (tidy < omega) or (tidx > input.shape[0]+(omega)) or (tidy > input.shape[1]+(omega)):
    return

  output[(tidx+omega), (tidy+omega), 0] = shared_memory[local_tidx, local_tidy, 0]
 


In [ ]:
@cuda.jit
def generatewindow(hsvinput, omega, shapeimage, output):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(16, 16, 1), dtype=np.float64)
  
  if (tidx < omega) or (tidy < omega) or (tidx > shapeimage[0]+(omega)) or (tidy > shapeimage[1]+(omega)):
    return

  if local_tidx < hsvinput.shape[0] and local_tidy < hsvinput.shape[1]:
    shared_memory[local_tidx, local_tidy, 0] = hsvinput[tidx-4, tidy-4, 0]
    shared_memory[local_tidx+8, local_tidy, 0] = hsvinput[tidx+4, tidy-4, 0]
    shared_memory[local_tidx, local_tidy+8, 0] = hsvinput[tidx-4, tidy+4, 0]
    shared_memory[local_tidx+8, local_tidy+8, 0] = hsvinput[tidx+4, tidy+4, 0]

  cuda.syncthreads()
  
  # if (tidx < omega) or (tidy < omega) or (tidx > shapeimage[0]+(omega)) or (tidy > shapeimage[1]+(omega)):
  #   return

  # if (local_tidx < omega) or (local_tidy < omega) or (local_tidx > 8+(omega)) or (local_tidy > 8+(omega)):
  #   return

  count = 0
  for wx in range(0, omega+1):
    for wy in range(0, omega+1):
      w1 = shared_memory[local_tidx+omega-wx,local_tidy+omega-wy,0]
      w2 = shared_memory[local_tidx+omega+wx,local_tidy+omega-wy,0]
      w3 = shared_memory[local_tidx+omega-wx,local_tidy+omega+wy,0]
      w4 = shared_memory[local_tidx+omega+wx,local_tidy+omega+wy,0]

      output[tidx,tidy,0,count] = w1 
      output[tidx,tidy,1,count] = w2
      output[tidx,tidy,2,count] = w3
      output[tidx,tidy,3,count] = w4
       
      count += 1

  cuda.syncthreads()

In [ ]:
@cuda.jit
def calstd(input, omega, output):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(4, 4, 4, 25), dtype=np.float64)

  size = (omega+1)**2

  if (tidx < omega) or (tidy < omega) or (tidx > shape[0]+omega) or (tidy > shape[1]+omega):
    return

  if local_tidx < input.shape[0] and local_tidy < input.shape[1]:
    for i in range(4):
      for j in range(size):
        shared_memory[local_tidx, local_tidy,i,j] = input[tidx, tidy,i,j]

  cuda.syncthreads()

  w0 = shared_memory[local_tidx, local_tidy,0]
  w1 = shared_memory[local_tidx, local_tidy,1]
  w2 = shared_memory[local_tidx, local_tidy,2]
  w3 = shared_memory[local_tidx, local_tidy,3]

  #Window 0
  t = 0
  for i in w0:
    t = t + i
  mean0 = t/size

  t2 = 0 
  for i in w0:
    t2 = t2 + (i- mean0)**2
  
  std0 = math.sqrt(t2/size)

  #Window 1
  t = 0
  for i in w1:
    t = t + i
  mean1 = t/size

  t2 = 0 
  for i in w1:
    t2 = t2 + (i- mean1)**2

  std1 = math.sqrt(t2/size)

  #Window 2
  t = 0
  for i in w2:
    t = t + i
  mean2 = t/size

  t2 = 0 
  for i in w2:
    t2 = t2 + (i- mean2)**2

  std2 = math.sqrt(t2/size)

  #Window 3
  t = 0
  for i in w3:
    t = t + i
  mean3 = t/size

  t2 = 0 
  for i in w3:
    t2 = t2 + (i- mean3)**2

  std3 = math.sqrt(t2/size)

  # minstd = min(std0, std1, std2, std3)
  
  minstd = 1
  for i in (std0,std1,std2,std3):
    if i < minstd:
      minstd = i

  if minstd == std0:
    output[tidx, tidy, 0] = 0
  elif minstd == std1:
    output[tidx, tidy, 0] = 1
  elif minstd == std2:
    output[tidx, tidy, 0] = 2
  elif minstd == std3:
    output[tidx, tidy, 0] = 3


In [ ]:
@cuda.jit
def addpadding2(input, output, omega):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(8, 8, 3), dtype=np.uint8)
  
  if local_tidx < input.shape[0] and local_tidy < input.shape[1]:
    shared_memory[local_tidx, local_tidy, 0] = input[tidx, tidy, 0]
    shared_memory[local_tidx, local_tidy, 1] = input[tidx, tidy, 1]
    shared_memory[local_tidx, local_tidy, 2] = input[tidx, tidy, 2]

  cuda.syncthreads()

  if (tidx < omega) or (tidy < omega) or (tidx > shape[0]+(omega)) or (tidy > shape[1]+(omega)):
    return

  output[(tidx+omega), (tidy+omega), 0] = shared_memory[local_tidx, local_tidy, 0]
  output[(tidx+omega), (tidy+omega), 1] = shared_memory[local_tidx, local_tidy, 1]
  output[(tidx+omega), (tidy+omega), 2] = shared_memory[local_tidx, local_tidy, 2]
 

In [ ]:
@cuda.jit
def kuwahara(input, stdwindow, shapeimage, omega, output):
  local_tidx = cuda.threadIdx.x
  local_tidy = cuda.threadIdx.y

  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

  shared_memory = cuda.shared.array(shape=(16, 16, 3), dtype=np.uint8)

  if (tidx < omega) or (tidy < omega) or (tidx > shapeimage[0]+(omega)) or (tidy > shapeimage[1]+(omega)):
    return

  if local_tidx < input.shape[0] and local_tidy < input.shape[1]:
    shared_memory[local_tidx, local_tidy, 0] = input[tidx-4, tidy-4, 0]
    shared_memory[local_tidx, local_tidy, 1] = input[tidx-4, tidy-4, 1]
    shared_memory[local_tidx, local_tidy, 2] = input[tidx-4, tidy-4, 2]

    shared_memory[local_tidx+8, local_tidy, 0] = input[tidx+4, tidy-4, 0]
    shared_memory[local_tidx+8, local_tidy, 1] = input[tidx+4, tidy-4, 1]
    shared_memory[local_tidx+8, local_tidy, 2] = input[tidx+4, tidy-4, 2]

    shared_memory[local_tidx, local_tidy+8, 0] = input[tidx-4, tidy+4, 0]
    shared_memory[local_tidx, local_tidy+8, 1] = input[tidx-4, tidy+4, 1]
    shared_memory[local_tidx, local_tidy+8, 2] = input[tidx-4, tidy+4, 2]

    shared_memory[local_tidx+8, local_tidy+8, 0] = input[tidx+4, tidy+4, 0]
    shared_memory[local_tidx+8, local_tidy+8, 1] = input[tidx+4, tidy+4, 1]
    shared_memory[local_tidx+8, local_tidy+8, 2] = input[tidx+4, tidy+4, 2]


  cuda.syncthreads()

  size = (omega+1)**2

  # if (local_tidx < omega) or (local_tidy < omega) or (local_tidx > 8+(omega)) or (local_tidy > 8+(omega)):
  #   return


  Rtotal = 0
  Gtotal = 0
  Btotal = 0
  for wx in range(omega+1):
     for wy in range(omega+1):
        if stdwindow[tidx, tidy,0]==0:
          Rtotal += shared_memory[local_tidx+omega-wx,local_tidy+omega-wy,0]
          Gtotal += shared_memory[local_tidx+omega-wx,local_tidy+omega-wy,1]
          Btotal += shared_memory[local_tidx+omega-wx,local_tidy+omega-wy,2]

          output[tidx, tidy, 0] = Rtotal/size
          output[tidx, tidy, 1] = Gtotal/size
          output[tidx, tidy, 2] = Btotal/size
       
        elif stdwindow[tidx, tidy,0]==1:
          Rtotal += shared_memory[local_tidx+omega+wx,local_tidy+omega-wy,0]
          Gtotal += shared_memory[local_tidx+omega+wx,local_tidy+omega-wy,1]
          Btotal += shared_memory[local_tidx+omega+wx,local_tidy+omega-wy,2]

          output[tidx, tidy, 0] = Rtotal/size
          output[tidx, tidy, 1] = Gtotal/size
          output[tidx, tidy, 2] = Btotal/size  

        elif stdwindow[tidx, tidy,0]==2:
          Rtotal += shared_memory[local_tidx+omega-wx,local_tidy+omega+wy,0]
          Gtotal += shared_memory[local_tidx+omega-wx,local_tidy+omega+wy,1]
          Btotal += shared_memory[local_tidx+omega-wx,local_tidy+omega+wy,2]

          output[tidx, tidy, 0] = Rtotal/size
          output[tidx, tidy, 1] = Gtotal/size
          output[tidx, tidy, 2] = Btotal/size

        elif stdwindow[tidx, tidy,0]==3:
          Rtotal += shared_memory[local_tidx+omega+wx,local_tidy+omega+wy,0]
          Gtotal += shared_memory[local_tidx+omega+wx,local_tidy+omega+wy,1]
          Btotal += shared_memory[local_tidx+omega+wx,local_tidy+omega+wy,2]

          output[tidx, tidy, 0] = Rtotal/size
          output[tidx, tidy, 1] = Gtotal/size
          output[tidx, tidy, 2] = Btotal/size
        
        # else:
        #   output[tidx, tidy, 0] = input[tidx, tidy, 0]
        #   output[tidx, tidy, 1] = input[tidx, tidy, 1]
        #   output[tidx, tidy, 2] = input[tidx, tidy, 2]

  cuda.syncthreads()


In [ ]:
blockSize = (8,8)
gridSize = (math.ceil(shape[0]/blockSize[0]),math.ceil(shape[1]/blockSize[1]))
omega = 4
image = cuda.to_device(im)


In [ ]:
t1 = time.time()

#conversion of RGB to HSV
hsvdev1 = cuda.device_array((shape[0],shape[1],1), np.float64)
HSVconversion[gridSize, blockSize](image, hsvdev1)

#add padding to HSV image
hsvdev2 = cuda.device_array((shape[0]+omega*2,shape[1]+omega*2,1), np.float64)
addpadding[gridSize, blockSize](hsvdev1, hsvdev2, omega)

#add generate window information
windowdev = cuda.device_array((shape[0]+omega*2,shape[1]+omega*2,4,(omega+1)**2), np.float64)
generatewindow[gridSize, blockSize](hsvdev2, omega, shape, windowdev)

#calclate STD and find lowest STD window
blockSize1 = (4,4)
gridSize1 = (math.ceil(shape[0]/blockSize1[0]),math.ceil(shape[1]/blockSize1[1]))
stdwindow = cuda.device_array((shape[0]+omega*2,shape[1]+omega*2,1), np.int8)
calstd[gridSize1, blockSize1](windowdev, omega, stdwindow)

#add padding to RGB image
impadded = cuda.device_array((shape[0]+omega*2,shape[1]+omega*2,3), np.uint8)
addpadding2[gridSize, blockSize](image, impadded, omega)

#apply filter
outputimage = cuda.device_array((shape[0]+omega*2,shape[1]+omega*2,3), np.uint8)
kuwahara[gridSize, blockSize](impadded, stdwindow, shape, omega, outputimage)

t2 = time.time()
print(t2-t1)

3.113365411758423


In [ ]:
outputimagehost = outputimage.copy_to_host()
imgpu = Image.fromarray(outputimagehost)
imgpu

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
imgpu.save("/content/drive/MyDrive/Colab Notebooks/kuwaharafilter_sharedmemGPU.jpeg")